# 第11章 指示チューニング

## 11.2 指示チューニングの実装

### 11.2.1 環境の準備

In [ ]:
!pip install datasets transformers[torch,sentencepiece] trl peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数シードを42に固定
set_seed(42)

### 11.2.2 データセットの準備

In [ ]:
from pprint import pprint
from datasets import load_dataset

# Hugging Face Hub上のリポジトリからデータセットを読み込む
dataset = load_dataset("llm-book/oasst1-21k-ja", split="train")
# データセットの形式と事例数を確認します。
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/21164 [00:00<?, ? examples/s]

Dataset({
    features: ['conversation'],
    num_rows: 21164
})


In [ ]:
from pprint import pprint

# pprintで見やすく表示する
pprint(dataset[0])

{'conversation': [{'content': 'こんにちは！', 'role': 'user'},
                  {'content': 'こんにちは！ご質問やお困りのことがありましたら、何でもご相談ください。何が必要か教えてください。',
                   'role': 'assistant'},
                  {'content': '世界のすべての国をアルファベット順に、それぞれの国の人口を教えてください。',
                   'role': 'user'},
                  {'content': '世界中の国をアルファベット順に並べたリストと、その国の推定人口です：\n'
                              '\n'
                              'アフガニスタン: 38,928,346 アルバニア: 2,877,797 '
                              'アルジェリア：44,344,744 アンドラ: 77,265 アンゴラ: 32,878,272 '
                              'アンティグア・バーブーダ: 97,929 アルゼンチン: 45,195,774 アルメニア: '
                              '2,977,600 オーストラリア: 25,499,884 オーストリア: 9,006,398 '
                              'アゼルバイジャン: 10,134,604 バハマ：393,248 バーレーン: '
                              '1,714,571 バングラデシュ: 164,689,383164,689,383 '
                              'バルバドス: 287,375 ベラルーシ: 9,449,323 ベルギー: '
                              '11,589,623 ベリーズ: 397,628 ベナン: 12,123,200 ブータン: 

### 11.2.3 チャットテンプレートの作成

In [ ]:
from transformers import AutoTokenizer

base_model_name = "tokyotech-llm/Swallow-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

In [ ]:
tokenizer.chat_template = """\
{%- for message in messages %}
{%- if message['role'] == 'user' %}
{{ bos_token + 'ユーザ：' + message['content'] + eos_token }}
{%- elif message['role'] == 'assistant' %}
{{ bos_token + 'アシスタント：'  + message['content'] + eos_token }}
{%- endif %}
{% if loop.last and add_generation_prompt %}
{{ bos_token + 'アシスタント：' }}
{%- endif %}
{% endfor %}\
"""

In [ ]:
# デフォルトではトークナイズかつID化されたトークンのリストが返されるが、
# ここではtokenize=Falseとしてトークナイズ前の文字列を返すように設定
chat_text = tokenizer.apply_chat_template(
    dataset[0]["conversation"], tokenize=False
)
# 発話間に改行が含まれないため、見やすくするためにeos_tokenを改行に置換
print(chat_text.replace(tokenizer.eos_token, "\n"))

<s>ユーザ：こんにちは！
<s>アシスタント：こんにちは！ご質問やお困りのことがありましたら、何でもご相談ください。何が必要か教えてください。
<s>ユーザ：世界のすべての国をアルファベット順に、それぞれの国の人口を教えてください。
<s>アシスタント：世界中の国をアルファベット順に並べたリストと、その国の推定人口です：

アフガニスタン: 38,928,346 アルバニア: 2,877,797 アルジェリア：44,344,744 アンドラ: 77,265 アンゴラ: 32,878,272 アンティグア・バーブーダ: 97,929 アルゼンチン: 45,195,774 アルメニア: 2,977,600 オーストラリア: 25,499,884 オーストリア: 9,006,398 アゼルバイジャン: 10,134,604 バハマ：393,248 バーレーン: 1,714,571 バングラデシュ: 164,689,383164,689,383 バルバドス: 287,375 ベラルーシ: 9,449,323 ベルギー: 11,589,623 ベリーズ: 397,628 ベナン: 12,123,200 ブータン: 754,288 ボリビア: 11,673,095 ボスニア・ヘルツェゴビナ: 3,279,000 ボツワナ：2,335,814 ブラジル: 212,559,417 ブルネイ: 441,484 ブルガリア: 7,034,879 ブルキナファソ: 20,903,273 ブルンジ: 11,890,781 カボベルデ: 555,987 カンボジア: 16,205,218 カメルーン: 26,545,863 カナダ：37,742,154 中央アフリカ共和国4,829,767 チャド: 16,425,864 チリ: 19,116,201 中国: 1,439,323,776 コロンビア: 50,882,891 コモロ: 869,601 コンゴ民主共和国：87,534,403 コンゴ共和国5,457,821 コスタリカ5,094,118 コートジボワール: 26,378,274 クロアチア: 4,105,267 キューバ: 11,239,224 キプロス：1,207,359 チェコ：10,708,919 デンマーク：5,792,2025,792,202



In [ ]:
# 会話データの末尾のアシスタントの発話を除き、生成を促すための文字列を追加
chat_text = tokenizer.apply_chat_template(
    dataset[0]["conversation"][:-1],
    tokenize=False,
    add_generation_prompt=True,
)
print(chat_text.replace(tokenizer.eos_token, "\n"))

<s>ユーザ：こんにちは！
<s>アシスタント：こんにちは！ご質問やお困りのことがありましたら、何でもご相談ください。何が必要か教えてください。
<s>ユーザ：世界のすべての国をアルファベット順に、それぞれの国の人口を教えてください。
<s>アシスタント：


### 11.2.4 トークンID への変換

In [ ]:
# チャットテンプレートを適用してトークンIDに変換
tokenized_dataset = [
    tokenizer.apply_chat_template(item["conversation"])
    for item in dataset
]
# トークンIDに変換されたデータセットの先頭を表示
token_ids = tokenized_dataset[0]
print("トークンID:", token_ids)
print("トークン:", tokenizer.convert_ids_to_tokens(token_ids))

トークンID: [1, 39944, 30383, 33328, 30584, 2, 1, 40135, 30383, 33328, 30584, 31622, 32916, 31111, 30697, 36451, 30199, 32002, 30458, 32009, 32006, 32075, 30330, 31502, 39682, 31622, 32277, 32059, 30267, 31502, 30458, 32090, 30412, 32903, 30466, 32059, 30267, 2, 1, 39944, 30383, 32198, 30199, 32668, 30199, 30356, 30396, 39042, 39383, 33045, 30353, 30330, 32728, 30199, 30356, 30199, 35620, 30396, 32903, 30466, 32059, 30267, 2, 1, 40135, 30383, 32198, 30275, 30199, 30356, 30396, 39042, 39383, 33045, 30353, 37231, 30366, 33634, 30364, 30330, 32016, 30356, 30199, 40875, 35620, 32001, 30383, 13, 13, 34874, 41714, 39676, 29901, 29871, 29941, 29947, 29892, 29929, 29906, 29947, 29892, 29941, 29946, 29953, 29871, 33997, 32873, 29901, 29871, 29906, 29892, 29947, 29955, 29955, 29892, 29955, 29929, 29955, 29871, 32513, 32882, 32166, 30383, 29946, 29946, 29892, 29941, 29946, 29946, 29892, 29955, 29946, 29946, 29871, 37329, 30281, 29901, 29871, 29955, 29955, 29892, 29906, 29953, 29945, 29871, 32709, 309

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
from trl import DataCollatorForCompletionOnlyLM

bos = tokenizer.bos_token
collator = DataCollatorForCompletionOnlyLM(
    instruction_template=bos + "ユーザ：",  # ユーザの発話開始を示す文字列
    response_template=bos + "アシスタント：",  # アシスタントの返答開始を示す文字列
    tokenizer=tokenizer,  # トークナイザ
)
# トークナイズされたデータセットの先頭をミニバッチ構築処理
batch = collator(tokenized_dataset[:1])
input_ids = batch["input_ids"][0]
labels = batch["labels"][0]
print("入力トークンID:", input_ids)
print("正解ラベル:", labels)

入力トークンID: tensor([    1, 39944, 30383, 33328, 30584,     2,     1, 40135, 30383, 33328,
        30584, 31622, 32916, 31111, 30697, 36451, 30199, 32002, 30458, 32009,
        32006, 32075, 30330, 31502, 39682, 31622, 32277, 32059, 30267, 31502,
        30458, 32090, 30412, 32903, 30466, 32059, 30267,     2,     1, 39944,
        30383, 32198, 30199, 32668, 30199, 30356, 30396, 39042, 39383, 33045,
        30353, 30330, 32728, 30199, 30356, 30199, 35620, 30396, 32903, 30466,
        32059, 30267,     2,     1, 40135, 30383, 32198, 30275, 30199, 30356,
        30396, 39042, 39383, 33045, 30353, 37231, 30366, 33634, 30364, 30330,
        32016, 30356, 30199, 40875, 35620, 32001, 30383,    13,    13, 34874,
        41714, 39676, 29901, 29871, 29941, 29947, 29892, 29929, 29906, 29947,
        29892, 29941, 29946, 29953, 29871, 33997, 32873, 29901, 29871, 29906,
        29892, 29947, 29955, 29955, 29892, 29955, 29929, 29955, 29871, 32513,
        32882, 32166, 30383, 29946, 29946, 29892, 2994

In [ ]:
import itertools

segments_to_fit: list[list[int]] = []
segments_to_ignore: list[list[int]] = []
# ラベルが-100である箇所とそうでない箇所ごとにグルーピング
for key, group in itertools.groupby(
    range(len(input_ids)), key=lambda i: labels[i] == -100
):
    group = list(group)
    if key:
        segments_to_ignore.append(group)
    else:
        segments_to_fit.append(group)

print("---- 損失を計算しない部分 ----")
for seg in segments_to_ignore:
    print(tokenizer.decode(input_ids[seg]))
    print()

print("---- 損失を計算する部分 ----")
for seg in segments_to_fit:
    print(tokenizer.decode(input_ids[seg]))
    print()

---- 損失を計算しない部分 ----
<s>ユーザ：こんにちは！</s><s>アシスタント：

<s>ユーザ：世界のすべての国をアルファベット順に、それぞれの国の人口を教えてください。</s><s>アシスタント：

---- 損失を計算する部分 ----
こんにちは！ご質問やお困りのことがありましたら、何でもご相談ください。何が必要か教えてください。</s>

世界中の国をアルファベット順に並べたリストと、その国の推定人口です：

アフガニスタン: 38,928,346 アルバニア: 2,877,797 アルジェリア：44,344,744 アンドラ: 77,265 アンゴラ: 32,878,272 アンティグア・バーブーダ: 97,929 アルゼンチン: 45,195,774 アルメニア: 2,977,600 オーストラリア: 25,499,884 オーストリア: 9,006,398 アゼルバイジャン: 10,134,604 バハマ：393,248 バーレーン: 1,714,571 バングラデシュ: 164,689,383164,689,383 バルバドス: 287,375 ベラルーシ: 9,449,323 ベルギー: 11,589,623 ベリーズ: 397,628 ベナン: 12,123,200 ブータン: 754,288 ボリビア: 11,673,095 ボスニア・ヘルツェゴビナ: 3,279,000 ボツワナ：2,335,814 ブラジル: 212,559,417 ブルネイ: 441,484 ブルガリア: 7,034,879 ブルキナファソ: 20,903,273 ブルンジ: 11,890,781 カボベルデ: 555,987 カンボジア: 16,205,218 カメルーン: 26,545,863 カナダ：37,742,154 中央アフリカ共和国4,829,767 チャド: 16,425,864 チリ: 19,116,201 中国: 1,439,323,776 コロンビア: 50,882,891 コモロ: 869,601 コンゴ民主共和国：87,534,403 コンゴ共和国5,457,821 コスタリカ5,094,118 コートジボワール: 26,378,274 クロアチア: 4,105,267 キューバ: 11,239,224 キプロス：1,207,3

### 11.2.5 QLoRA のためのモデルの準備

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# モデルの量子化の設定
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビット量子化のパラメータを読み込む
    bnb_4bit_quant_type="nf4",  # NF4量子化を使用
    bnb_4bit_compute_dtype=torch.bfloat16,  # 計算時のデータ型としてBF16を使用
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,  # 量子化設定
    use_cache=False,  # 後にgradient checkpointingを有効にするために必要
    device_map="auto",
)

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

#### LoRA

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

# LoRAの設定
peft_config = LoraConfig(
    r=128,  # 差分行列のランク
    lora_alpha=128,  # LoRA層の出力のスケールを調整するハイパーパラメータ
    lora_dropout=0.05,  # LoRA層に適用するドロップアウト
    task_type=TaskType.CAUSAL_LM,  # LLMが解くタスクのタイプを指定
    # LoRAで学習するモジュール
    target_modules=[
        # SwallowがベースとするLlamaModelクラスにおけるモジュール名
        # マルチヘッド注意機構のパラメータ
        # それぞれ「大規模言語モデル入門」2.2.4章における
        # W_q, W_k, W_v, W_o にあたる
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        # フィードフォワード層のパラメータ
        # 2.2.5章で解説されている通常のTransformerとやや異なり、
        # gate_projという追加のパラメータを持つ
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

model.enable_input_require_grads()  # 学習を行うために必要
model = get_peft_model(model, peft_config)  # モデルにLoRAを適用
model.print_trainable_parameters()  # 学習可能なパラメータ数を表示

trainable params: 319,815,680 || all params: 7,149,785,088 || trainable%: 4.4731


### 11.2.6 訓練の実行

In [ ]:
from google.colab import drive

# Google ドライブを"drive"ディレクトリ以下にマウント
drive.mount("drive")

Mounted at drive


In [ ]:
from transformers import Trainer, TrainingArguments

# 学習パラメータ
training_args = TrainingArguments(
    output_dir="./drive/MyDrive/llm_book/IT_results", # 結果の保存フォルダ
    bf16=True,  # BF16を使用した学習の有効化
    num_train_epochs=1,  # エポック数
    per_device_train_batch_size=2,  # 訓練時のバッチサイズ
    gradient_accumulation_steps=8,  # 勾配累積のステップ数（5.5.2）
    gradient_checkpointing=True,  # 勾配チェックポインティングの有効化（5.5.3）
    optim="paged_adamw_8bit",  # 最適化器
    learning_rate=3e-4,  # 学習率
    lr_scheduler_type="cosine",  # 学習率スケジューラの種類
    max_grad_norm=0.3,  # 勾配クリッピングにおけるノルムの最大値（9.4.3）
    warmup_ratio=0.1,  # 学習率のウォームアップの長さ（5.2.8）
    logging_steps=10,  # ロギングの頻度
    save_steps=300,  # モデルの保存頻度
    report_to="none",  # 外部ツールへのログを無効化
)

# 無料版のT4 GPUなど、低メモリ環境での学習パラメータ
# バッチサイズを小さく設定
# training_args = TrainingArguments(
#     output_dir="./drive/MyDrive/llm_book/IT_results", # 結果の保存フォルダ
#     bf16=True,  # BF16を使用した学習の有効化
#     num_train_epochs=1,  # エポック数
#     per_device_train_batch_size=1,  # 訓練時のバッチサイズ
#     gradient_accumulation_steps=4,  # 勾配累積のステップ数（5.5.2）
#     gradient_checkpointing=True,  # 勾配チェックポインティングの有効化（5.5.3）
#     optim="paged_adamw_8bit",  # 最適化器
#     learning_rate=3e-4,  # 学習率
#     lr_scheduler_type="cosine",  # 学習率スケジューラの種類
#     max_grad_norm=0.3,  # 勾配クリッピングにおけるノルムの最大値（9.4.3）
#     warmup_ratio=0.1,  # 学習率のウォームアップの長さ（5.2.8）
#     logging_steps=10,  # ロギングの頻度
#     save_steps=100,  # モデルの保存頻度
#     report_to="none",  # 外部ツールへのログを無効化
# )
# 学習時間を短縮する場合はデータセットを切り出し
# tokenized_dataset = tokenized_dataset[:1000]

trainer = Trainer(
    model,
    train_dataset=tokenized_dataset,  # トークンID化されたデータセット
    data_collator=collator,  # ラベルの加工及びミニバッチ構築処理を行うモジュール
    args=training_args,  # 訓練の設定
    tokenizer=tokenizer,  # パラメータ保存時にトークナイザも一緒に保存するために指定
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,2.041900
20,1.772400
30,1.566800
40,1.514300
50,1.413600
60,1.471700
70,1.482900
80,1.426000
90,1.455800
100,1.444200


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=1322, training_loss=1.2843762001363666, metrics={'train_runtime': 6638.9944, 'train_samples_per_second': 3.188, 'train_steps_per_second': 0.199, 'total_flos': 5.489260210553979e+17, 'train_loss': 1.2843762001363666, 'epoch': 0.9994329994329995})

In [ ]:
prompt = "LLMのチューニングが終わりました！お祝いのスピーチをお願いします。"
messages = [{"role": "user", "content": prompt}]
tokenized_chat = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)
generated_tokens = model.generate(tokenized_chat, max_new_tokens=512)
generated_text = tokenizer.decode(generated_tokens[0])
print(generated_text.replace(tokenizer.eos_token, "\n"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences betwee

<s>ユーザ：LLMのチューニングが終わりました！お祝いのスピーチをお願いします。
<s>アシスタント：こんにちは、皆さん！今日は待ちに待った日です。私たちの言語モデルのチューニングがついに終わりました！

私たちはここに集まり、この特別な日を祝います。私たちのAI 言語モデルは、私たちの生活に革命をもたらす可能性を秘めています。

チューニングが完了したことで、私たちのモデルはより高度になり、より正確になりました。私たちは、私たちの生活を改善し、より便利で効率的なものにするために、私たちのモデルを利用する方法について、多くの可能性を秘めた新しい世界に足を踏み入れました。


### 11.2.7 モデルの保存

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# 無料版Colab（T4 GPU）の場合はRAMの制限で量子化前のモデルを読み込めない場合があります
# その場合は以下のコードで、学習後のLoRAパラメータのみをアップロードすることが可能です

# model.push_to_hub("YOUR-ACCOUNT/Swallow-7b-hf-oasst1-21k-ja")

In [ ]:
from peft import PeftModel

# 学習したLoRA のパラメータを量子化していない学習前のモデルに足し合わせる
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
)
checkpoint_path = "./drive/MyDrive/llm_book/IT_results/checkpoint-1322"
tuned_model = PeftModel.from_pretrained(base_model, checkpoint_path)

# LoRA のパラメータのみをアップロードする場合は次の行をコメントアウト
tuned_model = tuned_model.merge_and_unload()

# Hugging Face Hub のリポジトリ名を指定
# "YOUR-ACCOUNT"は自らのユーザ名に置き換えてください
repo_name = "YOUR-ACCOUNT/Swallow-7b-hf-oasst1-21k-ja"
# トークナイザをアップロード
tokenizer.push_to_hub(repo_name)
# モデルをアップロード
tuned_model.push_to_hub(repo_name)

adapter_model.safetensors:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/llm-book/Swallow-7b-hf-qlora-oasst1-21k-ja/commit/c942e428a91ad04e09ff0c12d7f49550561abcc8', commit_message='Upload model', commit_description='', oid='c942e428a91ad04e09ff0c12d7f49550561abcc8', pr_url=None, pr_revision=None, pr_num=None)